In [16]:
# Importing the patstat client
from epo.tipdata.patstat import PatstatClient

# Initialize the PATSTAT client
patstat = PatstatClient()

# Access ORM
db = patstat.orm()
# Importing tables as models
from epo.tipdata.patstat.database.models import REG701_APPLN, REG741_APPLN_STATUS, REG707_PARTIES


In [12]:
q = db.query(
    REG701_APPLN.id,
    REG741_APPLN_STATUS.status_text
).join(
   REG741_APPLN_STATUS, REG701_APPLN.status == REG741_APPLN_STATUS.status
)

res = patstat.df(q)
res

,id,status_text
0,18759301,Unitary effect registered
1,21177871,Unitary effect registered
2,19155020,Unitary effect registered
3,19164056,Unitary effect registered
4,18844476,Unitary effect registered
...,...,...
20245,18813702,Request for unitary effect rejected
20246,19162343,Request for unitary effect rejected
20247,8016936,Request for unitary effect rejected
20248,19020506,Request for unitary effect rejected


Finding the different status values

In [26]:
q = db.query(
    REG741_APPLN_STATUS.status_text,
    REG741_APPLN_STATUS.status
)

res = patstat.df(q)
res

,status_text,status
0,Unitary effect registered,9
1,Request for unitary effect filed,6
2,Request for unitary effect withdrawn,4
3,Request for unitary effect rejected,5


Joining the tables

In [ ]:
q = db.query(
    REG701_APPLN.id,
    REG741_APPLN_STATUS.status_text
).join(
   REG741_APPLN_STATUS, REG701_APPLN.status == REG741_APPLN_STATUS.status
)
)
res = patstat.df(q)
res

Aggregating the different status values

In [9]:
from sqlalchemy import func

q = db.query(
    func.count(REG701_APPLN.id).label('applications'),
    REG741_APPLN_STATUS.status_text.label('status')
).join(
   REG741_APPLN_STATUS, REG701_APPLN.status == REG741_APPLN_STATUS.status
).group_by(
    REG741_APPLN_STATUS.status_text
)
res = patstat.df(q)
res

,applications,status
0,26,Request for unitary effect withdrawn
1,12,Request for unitary effect rejected
2,443,Request for unitary effect filed
3,19769,Unitary effect registered


Joining with the parties table

In [32]:
q = db.query(
    func.count(REG701_APPLN.id).label("Unitary Patents Registered"),
    REG707_PARTIES.name.label('Proprietor')
).join(
    REG707_PARTIES, REG701_APPLN.id == REG707_PARTIES.id
).filter(
    REG707_PARTIES.type == 'A',
    REG701_APPLN.status == 9
).group_by(
    REG707_PARTIES.name
).order_by(
    func.count(REG701_APPLN.id).desc()
)

res = patstat.df(q)
res

,Unitary Patents Registered,Proprietor
0,233,"Samsung Electronics Co., Ltd."
1,149,Telefonaktiebolaget LM Ericsson (publ)
2,136,Ethicon LLC
3,133,Siemens Aktiengesellschaft
4,129,QUALCOMM Incorporated
...,...,...
12299,1,Airinspace
12300,1,Innate S.r.l.
12301,1,"MacRae, Allan J."
12302,1,Pyromeral Systems
